In [1]:
%%capture --no-display
!pip install peft chromadb sentence_transformers pymorphy2 --upgrade 

# Описание решения

### Используемые данные и их предобработка

Для тестирования был сгенерирован случайным образом набор из 50 рейсов для городов: Москва, Казань, Санкт-Петербург, Чебоксары, Нижний Новгород, Сочи. Затем для каждого рейса был создан документ, содержащий данные из таблицы. Документы, содержащие город, заданный пользователем, подгружаются в промпт (если пользователь не упомянул город из рейсов в таблице, то подгружаются все документы).  

### Используемые технологии и модели

В качестве языковой модели используется <a href="https://huggingface.co/IlyaGusev/saiga_mistral_7b_lora">Сайга</a>. Эта модель используется для нахождения информации о рейсах по запросу пользователя.

В качестве модели для нахождения названия городов в сообщении пользователя используется предобученная на задаче NER модель <a href="https://huggingface.co/surdan/LaBSE_ner_nerel">LaBSE</a>.

В качестве модели для классификации токсичных комментариев используется обученная на русском датасете токсичных комментариевмодель <href="https://huggingface.co/sentence-transformers/LaBSE">BERT</a>

В качестве модели определения семантического сходства используется предобученная для задачи семантического сходства модель <href="https://huggingface.co/sentence-transformers/LaBSE">LaBSE</a>.

В качестве векторной базы данных, хранящей возможные темы сообщения пользователя используется ChromaDB.

### Алгоритм

1) Зарегестрировать пользователя

2) Принять сообщение пользователя и проверить на токсичность. В случае токсичного запроса ответ модели: "Переформулируйте пожалуйста Ваш запрос".

3) Сообщение пользователя соотнести к одной из тем: Покупка билета, Возврат билета, Узнать подробности о рейсах. Тема определяется как наиболее близкое предложение к запросу пользователя. Если нет достаточнно близкой темы, бот выводит: "Запрос не понятен. Попробуйте переформулировать."

 - В случае покупки и возврата билета ищется номер билета из бд в сообщение. Если номер рейса не найдет, модель уточняет номер рейса. Затем необходимо подтвердить действие.


 - В случае запроса информации о рейсах, сообщение вместе с документами о релевантных рейсах подается в языковую модель, после чего выводится ее ответ.


In [2]:
from random import randrange
from datetime import timedelta, datetime
import random
import time
import pandas as pd

def get_total_seconds(dt):
    return time.mktime(dt.timetuple())

def generate_random_dates(start, end, N):
    out = []
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    for i in range(N):
        random_second = randrange(int_delta)
        dt = start + timedelta(seconds=random_second)
        dt = datetime(dt.year, dt.month, dt.day, dt.hour)
        out.append(dt)
    return sorted(out, key = get_total_seconds)

def generate_random_cities(cities, N):
    out = []
    while len(out) < N:
        city1 = random.choice(cities)
        city2 = random.choice(cities)
        if city1 != city2:
            #out.append([city1, city2])
            out.append(f"{city1}, {city2}")
    return out
            
def generate_random_price(start, end, N):
    out = []
    for i in range(N):
        out.append(random.randrange(start, end) * 1000)
        
    return out
    

def generate_data(N = 50):
    cities =  ["Москва", "Казань", "Санкт-Петербург", "Чебоксары", "Нижний Новгород", "Сочи"]
    dates = generate_random_dates(datetime(2023, 12, 23), datetime(2023, 12, 28), N)
    cities = generate_random_cities(cities, N)
    prices = generate_random_price(10, 30, N)
    nums = list(range(10000, 10000 + N))
    free_seats = [random.random() > 0.25 for i in range(N)]
    
    df = pd.DataFrame()
    df['num'] = nums
    df['date'] = dates
    df['cities'] = cities
    df['price'] = prices
    df['free'] = free_seats
    
    return df
    
    
    
df = generate_data()
df

,num,date,cities,price,free
0,10000,2023-12-23 03:00:00,"Санкт-Петербург, Москва",15000,False
1,10001,2023-12-23 06:00:00,"Сочи, Казань",15000,False
2,10002,2023-12-23 08:00:00,"Чебоксары, Нижний Новгород",10000,False
3,10003,2023-12-23 12:00:00,"Санкт-Петербург, Сочи",27000,True
4,10004,2023-12-23 12:00:00,"Москва, Нижний Новгород",26000,True
5,10005,2023-12-23 15:00:00,"Казань, Нижний Новгород",12000,True
6,10006,2023-12-23 19:00:00,"Сочи, Нижний Новгород",29000,False
7,10007,2023-12-23 21:00:00,"Сочи, Москва",11000,False
8,10008,2023-12-23 23:00:00,"Сочи, Нижний Новгород",20000,False
9,10009,2023-12-24 01:00:00,"Чебоксары, Сочи",16000,True


In [3]:
def convert_df_to_docs(df):
    list_month = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
    docs = []
    for idx, row in df.iterrows():
        doc = f"Этот документ содержит информацию о рейсе №{df.values[idx, 0]}. "
        dt = df.values[idx, 1]
        price = df.values[idx, 3]
        free = df.values[idx, 4]
        city1, city2 = df.values[idx, 2].split(',')
        if free:
            free = "Со свободными местами (доступными билетами)"
        else:
            free = "Без свободных мест (без доступных билетов)"
        doc += f"Рейс {free} из {city1} в {city2} отправляется {dt.day} {list_month[dt.month-1]} {dt.year} года в {dt.hour} часов, с ценой {price} рублей.\n"
    
        docs.append(doc)
    return docs
    
docs = convert_df_to_docs(df)
docs[:5]

['Этот документ содержит информацию о рейсе №10000. Рейс Без свободных мест (без доступных билетов) из Санкт-Петербург в  Москва отправляется 23 декабря 2023 года в 3 часов, с ценой 15000 рублей.\n',
 'Этот документ содержит информацию о рейсе №10001. Рейс Без свободных мест (без доступных билетов) из Сочи в  Казань отправляется 23 декабря 2023 года в 6 часов, с ценой 15000 рублей.\n',
 'Этот документ содержит информацию о рейсе №10002. Рейс Без свободных мест (без доступных билетов) из Чебоксары в  Нижний Новгород отправляется 23 декабря 2023 года в 8 часов, с ценой 10000 рублей.\n',
 'Этот документ содержит информацию о рейсе №10003. Рейс Со свободными местами (доступными билетами) из Санкт-Петербург в  Сочи отправляется 23 декабря 2023 года в 12 часов, с ценой 27000 рублей.\n',
 'Этот документ содержит информацию о рейсе №10004. Рейс Со свободными местами (доступными билетами) из Москва в  Нижний Новгород отправляется 23 декабря 2023 года в 12 часов, с ценой 26000 рублей.\n']

In [4]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, BertTokenizer, BertForSequenceClassification
from transformers import pipeline

import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.utils import embedding_functions

import pymorphy2

class LLModel:
    def __init__(self, model_name = "IlyaGusev/saiga_mistral_7b_lora"):
        config = PeftConfig.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            config.base_model_name_or_path,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        self.model = PeftModel.from_pretrained(
            model,
            model_name,
            torch_dtype=torch.float16
        )
        self.model.eval()
        
        # Определяем токенайзер
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.generation_config = GenerationConfig.from_pretrained(model_name)
        
    def generate(self, prompt):
        data = self.tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
        data = {k: v.to(self.model.device) for k, v in data.items()}
        output_ids = self.model.generate(
            **data,
            generation_config=self.generation_config
        )[0]
        output_ids = output_ids[len(data["input_ids"][0]):]
        output = self.tokenizer.decode(output_ids, skip_special_tokens=True)
        return output.strip()
        
    def __call__(self, prompt):
        return self.generate(prompt).replace("bot", "")
    

class SentenceModel:
    def __init__(self, topics, model_name = "sentence-transformers/LaBSE", bd_name = "TopicBD"):
        self.client = chromadb.Client()
        self.bd_name = bd_name
        embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=model_name, device="cpu")
        self.collection = self.client.create_collection(bd_name, embedding_function = embedding_func)#, metadata={"hnsw:space": "cosine"})
        self.collection.add(
            documents = list(topics.values()),
            ids=[key for key in list(topics.keys())]
        )
        self.topics = topics
        
    def __del__(self):
        self.client.delete_collection(self.bd_name)
        
    def __call__(self, msg):
        res = self.collection.query(query_texts=[msg])
        if res['distances'][0][0] > 1.25:
            return None
        return res['ids'][0][0]
    


class NerExtractor:
    def __init__(self, model_checkpoint = "surdan/LaBSE_ner_nerel"):
        self.extractor = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="average")
    
    def __call__(self, text):
        results = self.extractor(text)
        list_city = []
        list_date = []
        
        city_end = -1000
        for res in results:
            entity = res['entity_group']
            
            if entity == "CITY":
                if city_end == res['start']: 
                    # Обработка слов через '-'
                    list_city[-1] = list_city[-1] + res['word']
                else:
                    list_city.append(res['word'])
                city_end = res['end']
                    
        return list_city
    

class BertClassifier:
    def __init__(self,model_path="chgk13/tiny_russian_toxic_bert",tokenizer_path="chgk13/tiny_russian_toxic_bert", device = 'cpu'):
        self.device = device
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device =device
        self.max_len = 512
        

    
    def __call__(self, text):
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:


class Manage():
    def __init__(self, df, docs, topics, prompt_template, ll_model, ner_model, topic_model, toxic_model):
        self.df = df
        self.docs = docs
        self.prompt_template = prompt_template
        self.topics = topics
        #self.ll_model = LLModel()
        #self.ner_model = NerExtractor()
        #self.topic_model = SentenceModel(topics)
        
        self.ll_model = ll_model
        self.ner_model = ner_model
        self.topic_model = topic_model
        self.toxic_model = toxic_model
        
        
        
        self.db = {} # хранит ФИО и список купленных билетов
        self.name =  None
        self.num_flight = None
        
        self.toxic_interpreter = ['neutral', 'toxic']
        self.fsm ={
            "registry": {"text" : "Введите ФИО: ", "proccesing": self.registry},
            "default": {"text": "Введите сообщение:", "proccesing": self.default},
            "buy": {"text":'''Подверждаете оформление (да/нет) покупки билета №{num} {title} {date}:''', "proccesing": self.buy},
            "refund": {"text":'''Подверждаете возврат (да/нет) билета №{num} {title} {date}:''', "proccesing": self.refund},
            "correction_buy": {"text": "Укажите номер рейса (например, №12345):", "proccesing": self.correction_buy},
            "correction_refund": {"text": "Укажите номер рейса (например, №12345):", "proccesing": self.correction_refund}
        }
        
    def _get_normal_form(self, word):
        word = "".join(word.split())
        morph = pymorphy2.MorphAnalyzer()
        return morph.parse(word)[0].normal_form
        
    def _is_digits(self, s):
        for c in s:
            if not("0" <=c and c <= "9"):
                return False
        return True

    def _is_num_flight(self, msg):
        for word in msg.split():
            word = word.replace("№", "")
            if self._is_digits(word) and int(word) in self.df.values[:, 0]: # оптимизировать
                self.num_flight = int(word)
                return True
        return False 
    
    def check_fio(self, s):
            if len(s.strip().split()) != 3:
                return False
            for char in s.lower():
                if not("я">=char>="а" or char == " "):
                    return False
            return True
    
    def _filter_docs(self, msg):
        out = []
        cities = self.ner_model(msg)
        for doc in self.docs:
            cities_in_doc = True
            for city in cities:
                city = self._get_normal_form(city)
                
                if not(city in doc.lower()):
                    cities_in_doc = False
                    break
            if cities_in_doc:
                out.append(doc)
                
        if len(out) == 0: # Значит нету в запросе города из рейса
            return self.docs
        else:
            return out
        
    def registry(self, msg):
        if len(msg.split()) != 3 or not(self.check_fio(msg.lower())):
            return "registry", "Ошибка в ФИО. Введите еще раз:"
        self.name = msg
        self.db[self.name] = []
        return "default", ""

    def default(self, msg):
        topic = self.topic_model(msg)
        if topic is None:
            return "default", "Не понятен ваш запрос. Попробуйте переформулировать."
        if topic == "buy":
            if self._is_num_flight(msg):
                return "buy", ""
            else:
                return "correction_buy", ""

        if topic == "refund":
            if self._is_num_flight(msg):
                return "refund", ""
            else:
                return "correction_refund", ""


        if topic == "about":
            _docs = self._filter_docs(msg)
            prompt = self.prompt_template.format(inp=msg, text = "".join(_docs), count = len(_docs))
            return "default", self.ll_model(prompt)


    def buy(self, msg):
        if msg.lower() == 'да':
            self.db[self.name].append(self.num_flight)
            return "default", "Покупка успешна совершена"
        else:
            return "default", "Отмена покупки"

    def refund(self, msg):
        if msg.lower() == 'да'and self.num_flight in self.db[self.name]:
            self.db[self.name].remove(self.num_flight)
            return "default", "Возврат успешно совершен"
        else:
            return "default", "Отмена возврата"

    def correction_buy(self, msg):
        if self._is_num_flight(msg):
            return "buy", ""
        else:
            return "default", "Некорректный номер рейса"

    def correction_refund(self, msg):
        if self._is_num_flight(msg):
            return "refund", ""
        else:
            return "default", "Некорректный номер рейса"
        
    def start(self):
        state = 'registry'
        while True:
            if state in ['buy', 'refund']:
                row = df.iloc[list(df['num'].values).index(self.num_flight)]
                text = f"{self.fsm[state]['text']}".format(num = row['num'], title = row['cities'], date = row['date'])
            else:
                text = f"{self.fsm[state]['text']}"
                
            msg = input(text)
            if msg == "/exit":
                break
            if self.toxic_interpreter[self.toxic_model(msg)] == "toxic":
                ans = "Пожалуйста переформулируйте Ваш запрос"
            else:
                state, ans = self.fsm[state]["proccesing"](msg)

            print(ans)

            


topics = {"buy":"Забронировать, купить", "refund":"Отменить, оформить возврат, вернуть", "about":"Рассказать какие доступные рейсы"}
PROMT_TEMPLATE = '''<s>system\nТы — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и отвечаешь на вопросы на русском языке.</s>\n
<s>user
Дай ответ пользователю, основываясь только на информации ниже: \n
Сегодня 22 декабря 2023 года. Текст содержит информацию об {count} рейсах на самолет. {text}
{inp}
</s> 
<s>bot'''



In [6]:
ll_model = LLModel()
ner_model = NerExtractor()
topic_model = SentenceModel(topics)
toxic_model = BertClassifier()

manage = Manage(df, docs, topics, PROMT_TEMPLATE, ll_model, ner_model, topic_model, toxic_model)

adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/47.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

In [17]:
manage.start()

Введите ФИО:  Дмитрий Иванович Петров


Введите сообщение: Подскажи на какой рейс из Казани в Москву я могу купить? 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Вы можете купить билет на рейс №10025, который отправляется из Казань в Москва 26 декабря 2023 года в 5 часам ночи. Рейс имеет свободные места (доступные билеты), и стоимость билета составляет 14 000 рублей.


Введите сообщение: купи билет на рейс 10025


...

Возврат успешно совершен


Введите сообщение: exit


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Не понятен ваш запрос. Попробуйте переформулировать.


Введите сообщение: /exit


In [18]:
df 

,num,date,cities,price,free
0,10000,2023-12-23 03:00:00,"Санкт-Петербург, Москва",15000,False
1,10001,2023-12-23 06:00:00,"Сочи, Казань",15000,False
2,10002,2023-12-23 08:00:00,"Чебоксары, Нижний Новгород",10000,False
3,10003,2023-12-23 12:00:00,"Санкт-Петербург, Сочи",27000,True
4,10004,2023-12-23 12:00:00,"Москва, Нижний Новгород",26000,True
5,10005,2023-12-23 15:00:00,"Казань, Нижний Новгород",12000,True
6,10006,2023-12-23 19:00:00,"Сочи, Нижний Новгород",29000,False
7,10007,2023-12-23 21:00:00,"Сочи, Москва",11000,False
8,10008,2023-12-23 23:00:00,"Сочи, Нижний Новгород",20000,False
9,10009,2023-12-24 01:00:00,"Чебоксары, Сочи",16000,True
